Results for:

In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion.brendel_bethge import BrendelBethgeAttack as BBA

import pandas as pd
import numpy as np

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

Define RL agent

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
import os
path = os.path.join(os.getcwd(), '..','observation_masks.csv')
path = os.path.normpath(path) #resolve '..'
observation_masks = pd.read_csv(path)

In [7]:
%%time
%%capture
attack = utils.define_attack(agent, env, BBA)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,mask=observation_masks['mask_time'].to_numpy())

CPU times: total: 8h 18min 20s
Wall time: 8h 18min 17s


In [8]:
asr

1.0

In [9]:
kpi

cost_function
annual_peak_average                      1.198347
carbon_emissions_total                   0.917995
cost_total                               0.830110
daily_one_minus_load_factor_average      1.412863
daily_peak_average                       1.005608
electricity_consumption_total            0.928516
monthly_one_minus_load_factor_average    0.997774
ramping_average                          1.331715
zero_net_energy                          1.109182
Name: District, dtype: float64

In [10]:
pd.DataFrame(obs).to_csv('bb results/clean obs.csv')
pd.DataFrame(adv_obs).to_csv('bb results/bb obs.csv')
pd.DataFrame(kpi).to_csv('bb results/bb kpi.csv')